In [5]:
import numpy as np
import pandas
import os

In [190]:
def csv2txt(csv):
    file = []
    with open(csv, 'r') as f: 
        for line in f.readlines():
            file.append(line)
    mx = 0
    key = 0
    for i in file:
        if ('Key_signature' in i):
            key = int(i.split(',')[3])
        if ('End_track' in i):
            length = int(i.split(',')[1])
            break
    everything = np.zeros((16, int(length/256) + 1))
    active = np.zeros(16)
    previous = (0,0,0,0,0,0)
    for i in file[8: -2]:
        if ('Key_signature' in i):
            key = int(i.split(',')[3].strip())
        if (len(i.split(',')) == 6):
            track, time, on, channel, note, velocity = i.strip().split(',')
            time = int(int(time)/256)
            track = int(track)
            channel = int(channel)
            note = int(note) 
            note -= int(key)
            mx = max(mx, note)
            velocity = int(velocity)
            vec = (track, time, on, channel, note, velocity)
            if (on == ' Note_off_c'):
                for i in range(np.int(active[channel]), time + 1):
                    everything[channel, i] = note
            elif (on == ' Note_on_c'):
                active[channel] = time
            previous = vec
    everything = np.delete(everything, np.argwhere(everything.sum(axis = 1) == 0), axis = 0)
    st = ''
    for i in everything.T:
        for j in i:
            if (j != 0):
                st += chr(int(j) + 34)
            else:
                st += chr(34)
        st += ' '
    return st

In [12]:
def str2csv(st, f_name):
    if(st[-1] != ' '):
        st += ' '
    noc = 0
    for inst in st.split(' '):
        noc = max(len(inst), noc)
    noc = (min(noc, 15))
    song = np.ones((noc, st.count(' ')))*34
    count = 0
    for inst in st.split(' '):
        count2 = 0
        for note in inst:
            song[count2, count] = ord(note) 
            count2 +=1
        count += 1
    file = ['0, 0, Header, 1, %d, 1024\n' % (song.shape[0] + 1), 
        '1, 0, Start_track\n',
        '1, 0, Time_signature, 4, 2, 24, 8\n',
        '1, 0, Key_signature, 0, "major"\n',
        '1, 0, Tempo, 750000\n',
        '1, ' + str(count * 256)  + ', End_track\n']
    for i in range(song.shape[0]):
        curr = 34
        file.append("%d, %d, %s\n" % (i+2, 0, "Start_track"))
        file.append("%d, %d, %s, %s %d\n" % (i+2, 0, "Title_t", "Instrument", i + 1))
        file.append("%d, 0, Control_c, 0, 7, 101\n" % (i+2))
        file.append("%d, 0, Control_c, 0, 10, 64\n" % (i+2))
        for j in range(song.shape[1]):

            if (song[i][j] != curr):
                if (curr != 34):   
                    file.append("%d, %d, %s, %d, %d, %d\n" % (i + 2, j * 256, 'Note_off_c', i, curr - 34, 0))
                file.append("%d, %d, %s, %d, %d, %d\n" % (i + 2, j * 256, 'Note_on_c', i, song[i][j] - 34, 96))
                curr = song[i][j]

        file.append("%d, %d, %s\n" % (i+2, (song.shape[1] + 1) * 256, "End_track"))
    file.append("0, 0, End_of_file")
    f = open(f_name, "w")
    for i in file:
        f.write(i)    
    f.close()

In [191]:
f = open("complete_bach.txt", "w")   
for filename in os.listdir("datac"):
    if (filename.endswith(".csv")):
        f.write(csv2txt("datac/" + filename))
        f.write('\n')
f.close()

223.csv
237.csv
155.csv
141.csv
169.csv
97.csv
83.csv
196.csv
68.csv
182.csv
6.csv
54.csv
40.csv
431.csv
357.csv
343.csv
425.csv
419.csv
394.csv
380.csv
381.csv
395.csv
418.csv
342.csv
424.csv
430.csv
356.csv
41.csv
7.csv
55.csv
183.csv
197.csv
69.csv
82.csv
168.csv
96.csv
140.csv
154.csv
236.csv
222.csv
234.csv
220.csv
208.csv
142.csv
156.csv
80.csv
94.csv
181.csv
195.csv
43.csv
57.csv
5.csv
426.csv
340.csv
354.csv
432.csv
368.csv
383.csv
397.csv
396.csv
382.csv
369.csv
355.csv
433.csv
427.csv
341.csv
56.csv
4.csv
42.csv
194.csv
180.csv
95.csv
81.csv
157.csv
143.csv
209.csv
221.csv
235.csv
219.csv
231.csv
225.csv
85.csv
91.csv
147.csv
153.csv
46.csv
0.csv
52.csv
184.csv
190.csv
379.csv
345.csv
423.csv
437.csv
351.csv
386.csv
392.csv
393.csv
387.csv
436.csv
350.csv
344.csv
422.csv
378.csv
191.csv
185.csv
1.csv
53.csv
47.csv
152.csv
146.csv
90.csv
84.csv
224.csv
230.csv
218.csv
226.csv
232.csv
92.csv
178.csv
86.csv
150.csv
144.csv
51.csv
3.csv
45.csv
193.csv
187.csv
79.csv
408.csv
352.c

In [7]:
str2csv("Whka hc_\\O da]\\ da]Z da\\X da\\X da\\Z da\\N kc\\P kc\P md\\P kd\\P kd\\P kd\\P kd\\P kd\\P fd]P dd\\Q dd]Q d]ZN d ", "create.csv")


In [8]:
str2csv("Wh` h`WU hcYS hcZS hcZS haZS haXQ haXQ haXQ h_XP h_ZW h_XX h_XX h_XX h_XX h_XX h_XX h_XX h_XX j_XX jaX ", "create2.csv")


In [9]:
str2csv("Wh hbXUH haYUu haYU haYU haYU haYU haYU hdaU hd_U hdaU hdaU hdaU faWS faWK faXU faXS faXQ faXQ faXQ fa ", "create3.csv")


In [13]:
str2csv('+YWW _\WW c_\W c`XW c`WW `cYY c`YY b`YY b`YY eb^Y eb^Y gc`[ gc`[ gc`[ gb`[ o``[ oc`[ qc`Y me`Y qe`Y qe`Y qe`Y qe`Y le`Y e`]Y ^^YR g^WT ^[W[ ^[W[ `^[W `[[X `][X `^[X `]ZY c`[X c`[Y eb^T eb^V eb^V eb^V eb^V eb^V eb^V eb^V eb^V gc^V gb^ gl^ nb^ lg^T le[T le[T le]T le]T le]T qe]T ee]T gc]T gc^T eb]T gb[R b[[R b[[" b[[" b["[ b[[" b]]" kb]" bg[" kb[" kbY" i`W" lgW" lcT" lcT" lc"Q le`P leb^ leb^ leb^ lee', "create4.csv")
        